In [1]:
import os

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
import tensorflow as tf
import numpy as np

2023-04-13 00:00:06.901243: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
ROOT_PATH = '/'.join(os.getcwd().split('/')[:-3])
DATA_PATH = f'{ROOT_PATH}/dataset/NLP/wonderland.txt'
DATA_PATH

'/home/jovyan/TIL/dataset/NLP/wonderland.txt'

In [3]:
text = open(DATA_PATH, 'r', encoding = 'utf-8').read()
text = text.lower()

In [4]:
chars    = sorted(list(set(text)))
char2idx = dict((c, i) for i, c in enumerate(chars))

n_chars  = len(text)
n_vocab  = len(chars)

n_chars, n_vocab

(145421, 53)

In [5]:
seq_length     = 100
data_x, data_y = [], []

for idx in range(0, n_chars - seq_length, 1):
    
    seq_in  = text[idx : idx + seq_length]
    seq_out = text[idx + seq_length]
    
    data_x.append([char2idx[char] for char in seq_in])
    data_y.append(char2idx[seq_out])
    
n_patterns = len(data_x)
f'Total patterns : {n_patterns:,}'

'Total patterns : 145,321'

In [6]:
data_x = np.reshape(data_x, (n_patterns, seq_length, 1))
data_x = data_x / float(n_vocab) 
data_y = to_categorical(data_y)

In [7]:
model = Sequential()
model.add(LSTM(256, input_shape = (data_x.shape[1], data_x.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(data_y.shape[1], activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

In [8]:
model_path = f'{os.getcwd()}/model'
os.makedirs(model_path, exist_ok = True)

checkpoint = ModelCheckpoint(f'{model_path}/best_weight.hdf5', monitor = 'loss', verbose = 1,
                             save_best_only = True, mode = 'min')
callbacks  = [checkpoint]

In [9]:
model.fit(data_x, data_y, epochs = 100, batch_size = 128, callbacks = callbacks)

Epoch 1/100
1136/1136 [==============================] - ETA: 0s - loss: 3.0205
Epoch 1: loss improved from inf to 3.02053, saving model to /home/jovyan/TIL/AI_study/basic/Tensorflow/model/best_weight.hdf5
1136/1136 [==============================] - 248s 217ms/step - loss: 3.0205
Epoch 2/100
1136/1136 [==============================] - ETA: 0s - loss: 2.8305
Epoch 2: loss improved from 3.02053 to 2.83045, saving model to /home/jovyan/TIL/AI_study/basic/Tensorflow/model/best_weight.hdf5
1136/1136 [==============================] - 236s 208ms/step - loss: 2.8305
Epoch 3/100
1136/1136 [==============================] - ETA: 0s - loss: 2.7430
Epoch 3: loss improved from 2.83045 to 2.74303, saving model to /home/jovyan/TIL/AI_study/basic/Tensorflow/model/best_weight.hdf5
1136/1136 [==============================] - 234s 206ms/step - loss: 2.7430
Epoch 4/100
1136/1136 [==============================] - ETA: 0s - loss: 2.6704
Epoch 4: loss improved from 2.74303 to 2.67042, saving model to /h

In [10]:
idx2char = {i : c for c, i in char2idx.items()}
open_Quotes     = idx2char[51]

In [11]:
test_sequence = f'A dove and a bird of unknown species were eating pizza. Then the dove spoke. {open_Quotes}Come to think of it,  '.lower()
test_indexes  = [char2idx[seq] for seq in test_sequence]

In [12]:
result_sequence = test_sequence

for _ in range(1000):
    x = np.reshape(test_indexes, (1, len(test_indexes), 1))
    x = x / float(n_vocab)

    prediction    = model.predict(x, verbose = 0)
    index         = np.argmax(prediction)
    result        = idx2char[index]
    sequence      = [idx2char[test_index] for test_index in test_indexes]

    test_indexes.append(index)
    test_indexes  = test_indexes[1: len(test_indexes)]
    
    result_sequence += result

result_sequence

'a dove and a bird of unknown species were eating pizza. then the dove spoke. “come to think of it,  she pooe aad no it was op sinee in the daresn she was a ait oo tte oe tee of them say hiw fne and so soee on the shape. \nalice dould net eead on a lor to sie white rab it haa beel eoone ti the gorre of the garten, and the tere thiig then in was gosd toeetiig  the was aol the rurerss are tuiering  the was aoling dorn hn a lelale of petert \n“hom tome to mooe in on,” shiu hlir alicl, “that soee to taek to the poo, to she set oo lonk thet wire an ooeers aaret ht madreng it the was aown on the thing of the white rabbit, and tele teat sore the tools oo the tai oo the tabeit herself of the gousten and the war no the table. but it had nott beme her head to hend thet she was a cittorl oo the rabbit soire  bhtnee hor aami to tee and teit so say shem the was aowtoing, and the weit dot afain, and was going to betin at the pame of the war so sers ceandengn would belne so see it why: tie sas aowthe